In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
import warnings

import scalp
from scalp.output import draw
import lmz
import numpy as np



In [ ]:

if True:
    import better_exceptions
    import sys
    ip = get_ipython()
    def custom_exc(shell, etype, evalue, tb, tb_offset=None):
        better_exceptions.excepthook(etype, evalue, tb)
    ip.set_custom_exc((Exception,), custom_exc)


warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", module="anndata")

In [ ]:
scalp.test_config.scib_datapath

In [ ]:
#datasets = scalp.data.loaddata_scib(scalp.test_config.scib_datapath,  maxdatasets=3,  maxcells = 1000 )

datasets = list(scalp.data.scib(scalp.test_config.scib_datapath,  maxdatasets=3,  maxcells = 300 ))
dataset = datasets[-1] # this is the pancreatic

# scanorama

In [ ]:
import scanpy as sc

'''
def scanorama(dataset, dim = 2):
    dataset = scalp.pca.pca(dataset)
    dataset = scalp.mnn.scanorama(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='scanorama', dim= dim)
    return scalp.transform.stack(dataset)
st = scanorama(dataset)
sc.pl.umap(st, color=['batch', 'label'])
'''

st = scalp.mnn.scanorama(dataset) 
scalp.plot(st,'scanorama', color=['batch','label'])


# UMAP ONLY 

In [ ]:
import scanpy as sc



'''
def umaponly(dataset,dim=2):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='pca40', dim =dim)
    return scalp.transform.stack(dataset)
stack = umaponly(dataset)
sc.pl.umap(stack, color=['batch', 'label'])
'''  


# st = scalp.mnn.umaponly(dataset, dim=2)
# st.obsm['umaponly'] = st.obsm['']
scalp.plot(st,'pca40', color=['batch','label'])

In [ ]:
import scanpy as sc

'''
def bbknn(dataset, dim = 2):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    dataset = scalp.mnn.bbknnwrap(dataset, dim = dim)
    stack = scalp.transform.stack(dataset)
    return stack

stack = bbknn(dataset)
sc.pl.umap(stack, color=['batch', 'label'])
'''

st = scalp.mnn.bbknnwrap(dataset, dim =2) 
scalp.plot(st,'bbknn', color=['batch','label'])

# Combat 

In [ ]:
import scanpy as sc
'''
def combat(dataset, dim = 2):
    [ds.obsm.pop('umap', None)for ds in dataset]
    dataset = scalp.mnn.combat(dataset)
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='combat', dim =dim)
    return scalp.transform.stack(dataset)
stack= combat(dataset)
sc.pl.umap(stack, color=['batch', 'label'])
'''


st = scalp.mnn.combat(dataset) 
scalp.plot(st,'combat', color=['batch','label'])

# SCALP

In [ ]:
scalp.plot(st, color=['batch','label'])

In [ ]:
import scanpy as sc
from scalp import graph as sgraph
from scipy.sparse import csr_matrix

def Scalp(dataset, dim = 2):
    parm = {'neighbors_total': 60, 'intra_neighbors_mutual': False,
            'neighbors_intra_fraction': .33, 'add_tree': False, "epsilon":-1,
                  'copy_lsa_neighbors': False, 'horizonCutoff':0, # this hz cutoff is what we look at 
            'inter_outlier_threshold': -1, 'distance_metric':'euclidean', 'standardize':0,
            'inter_outlier_probabilistic_removal': False}


    bestparm = {'neighbors_total': 27,
         'neighbors_intra_fraction': 0.2204608669516461,
         'inter_outlier_threshold': 0.7495085614413425,
         'inter_outlier_probabilistic_removal': 0,
         'intra_neighbors_mutual': 1,
         'copy_lsa_neighbors': 0,
         'add_tree': 0,
         'horizonCutoff': 60,
         'distance_metric': 'euclidean',
         'standardize': 0}
    # parm = {'add_tree': True, 'copy_lsa_neighbors': False, 'inter_outlier_probabilistic_removal': False,  'inter_outlier_threshold': 0.72, 'intra_neighbors_mutual': False, 'neighbors_intra_fraction': 0, 'neighbors_total': 1}
    grap = scalp.mkgraph(dataset,**bestparm)
    # neggraph = sgraph.negstuff(dataset,**parm)
    # scalp.umapwrap.graph_jax(dataset,(csr_matrix(grap), csr_matrix(neggraph)),label = 'umap', n_components = 2)
    print("graphdone")
    plt.matshow(grap.todense())
    plt.colorbar()
    
    projection  = scalp.umapwrap.umap_last_experiment(dataset,grap,label = 'umap', n_components = dim)
    # scalp.umapwrap.graph_pacmap(dataset,csr_matrix(grap).toarray(),label = 'umap', n_components = dim)
    # scalp.umapwrap.graph_umap(dataset,grap,label = 'umap', n_components = dim) # THIS IS WHAT WE WANT TO USE I GUESS
    dataset.obsm['scalp']= projection
    return dataset

stack = Scalp(dataset)
scalp.plot(stack,'scalp', color=['batch','label'])

In [ ]:
dataset

In [ ]:
import ubergauss.tools as ut
from lmz import *

funcs = [scanorama, umaponly, bbknn, combat, Scalp] 

fuid = Range(funcs)
dataid = Range(datasets) 
tasks = [(f,d) for f in fuid for d in dataid]

def run(fd):
    f,d = fd
    fun = funcs[f]
    dat = datasets[d]
    stack = fun(dat,8)
    return stack
    
mydata = ut.xxmap(run, tasks) 
for e in mydata:
    if 'umap' not in e.obsm:
        e.obsm['umap']= e.obsm['X_umap']
        e.obsm.pop('X_umap')

In [ ]:
# OK HERE COMES THE NEW SYSTEM
# lets make mydata compatible with the new system 

# tasks = function, dataid
datasets_stack = Map(scalp.transform.stack, datasets) 
for (fu,da), result in zip(tasks, mydata):
    datasets_stack[da].obsm[str(fu)] = result.obsm['umap']
    datasets_stack[da].uns.setdefault('methods', []).append(str(fu))




In [ ]:
datasets_stack[0].uns['methods']

In [ ]:
scores = { str(i): scalp.score.scalp_scores(datasets_stack[i]) for i in lmz.Range(datasets_stack) }

In [ ]:
scores

In [ ]:
funcs

In [ ]:
ranktable, dom = scalp.output.score.pareto_avg(scores)
ranktable

In [ ]:
dom

In [ ]:
from scalp.output.score import score_lin, score_lin_batch, score_scib_metrics
scoredics_lb = ut.xxmap(score_lin_batch, mydata)
scoredics_scib = ut.xxmap(score_scib_metrics, mydata)
scoredics_l = ut.xxmap(score_lin, mydata)

# score_lin_batch and score_lin -> pareto comparison

In [ ]:
import pandas as pd
import seaborn as sns
# this only applies if we dont use the scib score fucntion
funcs = 'scanorama, umaponly, bbknn, combat, Scalp'.split(', ')

results = [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'label' } for s,(f,d) in zip(scoredics_l,tasks)]
results += [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'batch'} for s,(f,d) in zip(scoredics_lb,tasks)]
df = pd.DataFrame(results)
sns.barplot(data=df, y = 'score', x = 'method', errorbar = 'sd', hue='target')
plt.show()
ours = df.pivot_table(index='method', columns='target', values='score')

In [ ]:
import lmz
# this is for SCIB scoring 


# turn scores into a dataframe
funcs = 'scanorama, umaponly, bbknn, combat, Scalp'.split(', ')
results = [ [{"method":funcs[f], 'score':ss, 'dataset':d, 'metric':scrmeth } 
             for (scrmeth,ss) in s.items() ]for s,(f,d) in zip(scoredics_scib,tasks)]
results = lmz.Flatten(results)
df = pd.DataFrame(results)



sns.barplot(data=df, y = 'score', x = 'method', errorbar = 'sd', hue = 'metric')
plt.legend(loc='right', bbox_to_anchor=(1.85, 0.5), ncol=1)
plt.show()

In [ ]:

def split_scib_scores(dicts):
    '''splits scores in batch and label scores'''
    batchwords = 'PCR_batch ASW_label/batch graph_conn'.split()
    
    def split(d):
        b = np.mean([v for k,v in d.items() if k in batchwords ])
        a = np.mean([v for k,v in d.items() if not k in batchwords ])
        return a,b
        
    scores = lmz.Map(split, dicts) 
    return lmz.Transpose(scores)    
  

scr_l, scr_b = split_scib_scores(scoredics_scib)
results = [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'bioconservation_scib_avg' } for s,(f,d) in zip(scr_l,tasks)]
results += [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'batch_scib_avg'} for s,(f,d) in zip(scr_b,tasks)]
df = pd.DataFrame(results)
sns.barplot(data=df, y = 'score', x = 'method', errorbar = 'sd', hue='target')
plt.show()
theirs = df.pivot_table(index='method', columns='target', values='score')

In [ ]:

our_score = [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'label' } for s,(f,d) in zip(scoredics_l,tasks) ]
our_score += [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'batch'} for s,(f,d) in zip(scoredics_lb,tasks)  ]
df2 = pd.DataFrame(our_score)
df2

In [ ]:
from ubergauss.optimization import pareto_scores
pareto_scores(df)

In [ ]:
pareto_scores(df2) # this is pancreatic only  btw 

In [ ]:
df

In [ ]:
results

In [ ]:
np.corrcoef(ours.batch.values, theirs.batch_scib_avg.values)[0,1], np.corrcoef(ours.label.values, theirs.bioconservation_scib_avg.values)[0,1]

In [ ]:
dom

avg_rank = dom.groupby('dataset').domcount.rank(pct=True).groupby(dom.method).mean()

avg_rank

In [ ]:
dom

In [ ]:
funcs

In [ ]:
import scikit_posthocs as sp
test_results = sp.posthoc_conover_friedman(
    dom,
    melted=True,
    block_col='dataset',
    group_col='method',
    y_col='domcount',
)
#sp.sign_plot(test_results)
plt.figure(figsize=(10, 2), dpi=100)
plt.title('Critical difference diagram of average score ranks')
sp.critical_difference_diagram(avg_rank, test_results)

In [ ]:
d={4:99}
d.setdefault(4,5)
print(d)

In [ ]:
# horizoncut test 
d = np.random.rand(10,10)
co = 2
print(np.partition(d, co, axis=1)[:, co])
print(np.sort(d,axis=1))

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
random_matrix = np.random.rand(10, 10)
mask = np.random.rand(10, 10) > 0.5
testmat = random_matrix * mask

testmat